In [ ]:
# Standard libraries of Python

from time import gmtime

# Dependencies
import pandas as pd
import requests
import re
from pandas import DataFrame
from bs4 import BeautifulSoup
from dataclasses import dataclass
from selectolax.parser import HTMLParser
from tqdm import tqdm

from backend.database.src import src
from backend.database.src.parse import process_hidden_ability,process_ability,process_form_ability

In [ ]:
@dataclass
class Pokemon:
    name: str
    gender: str
    types: list

In [ ]:
pokedex = []
url = 'https://www.serebii.net/pokedex-sm/{}.shtml'.format(str(1).zfill(3))
html = requests.get(url).text
tree = HTMLParser(html)
pokemon = Pokemon(name=tree.css_first('#content > main > div > div > table:nth-child(7) > tbody > tr:nth-child(2) > td:nth-child(1)').text(),
            gender=tree.css_first('#content > main > div > div > table:nth-child(7) > tbody > tr:nth-child(2) > td:nth-child(4)').text(),
            types=list(map(lambda x: x.attributes['alt'], tree.css('#content > main > div > div > table:nth-child(5) > tbody > tr:nth-child(2) > td.cen >> img'))))
pokedex.append(pokemon)

pd.DataFrame(pokedex)

In [ ]:
# Ejemplo de uso
html_text = requests.get(f'https://www.serebii.net/pokedex-sm/001.shtml').text
soup = BeautifulSoup(html_text, 'html.parser')
all_divs = soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[0].find_all('td', {'class': 'fooevo'})

In [ ]:
ala = foo_info.find_all('b')

abilities = {'ability': [], 'hidden_ability': []}
form_abilities = {'ability': [], 'hidden_ability': []}
skip_next_flag = False
form_abilities_flag = False

for tag in ala:
    if skip_next_flag:
        skip_next_flag = False
        continue

    if "Hidden Ability" in tag.text:
        skip_next_flag = process_hidden_ability(tag, abilities, form_abilities, skip_next_flag, form_abilities_flag)
    else:
        form_abilities_flag = process_form_ability(tag, form_abilities_flag)
        process_ability(tag, abilities, form_abilities, form_abilities_flag)

form_abilities_flag = False

In [ ]:
#Weakness
types = []

for td_tag in foo_info:
    a_tag = td_tag.find('img')
    if a_tag:
        type_text = a_tag['alt']
        types.append(type_text)

s = 'Attacking Move Type: ','-type'
for string in s:
    types = list(map(lambda x: x.replace(string,''),types))

print(types)

filtered = [tag for tag in foo_info if '*' in tag.get_text(strip=True)]
filtered_t = [i.text for i in filtered]
val = list(map(lambda x: int(x.replace('*','')),filtered_t))

In [ ]:
all_divs = soup.find_all('div', attrs={'align': 'center'})

In [ ]:
x = src.Pokemon(7,6)

In [ ]:
x._weakness

In [ ]:
all_divs = x.soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[0].find_all('td', {'class': 'fooinfo'})

In [ ]:
bases = all_divs[0].find('td', string=re.compile("Base Stats - Total.*")).find_next_siblings('td')

In [ ]:
all_divs[0]

In [ ]:
# Supongamos que 'x' es tu objeto que contiene el BeautifulSoup y 'all_divs' es la lista de divs
mega_evolution_line = None

# Buscar la línea que contiene la información sobre Mega Evolution
for div in all_divs:
    mega_evolution_td = div.find('td', {'class': 'fooevo', 'colspan': '6'})
    
    if mega_evolution_td:
        mega_evolution_line = mega_evolution_td
        break

# Verificar si se encontró la línea de Mega Evolution
if mega_evolution_line:
    # Ahora puedes obtener el contenido que está debajo de esa línea
    content_below_mega_evolution = mega_evolution_line.find_next_siblings()
    
    # Imprimir el contenido
    for element in content_below_mega_evolution:
        print(element)
else:
    print("No se encontró información sobre Mega Evolution.")

In [ ]:
tables = all_divs[0].find_all('table', {'class': 'dextable'})

In [ ]:
for p,table in enumerate(tables):
    print(p)
    print(table.find_all('td', {'class': 'fooevo', 'colspan': '6'}))

In [ ]:
# Buscar todas las tablas en el primer div
tables = all_divs[0].find_all('table', {'class': 'dextable'})

# Buscar la posición exacta de la línea con <td class="fooevo" colspan="6">
mega_evolution_position = -1
for i, table in enumerate(tables):
    if table.find('td', {'class': 'fooevo', 'colspan': '6'}):
        mega_evolution_position = i
        print(i)
        break

In [ ]:
k = tables[19]

In [ ]:
tables[19].find_all('td', class_='fooinfo')[5]